In [1]:
import squidpy as sq
import scanpy as sc
import pandas as pd 
import numpy as np 
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from adipo_finder import utils as seg_utils
from adipo_finder import segmentation as seg
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import adipo_finder.new_adipofinder_code as nac
from skimage import measure, morphology, segmentation, feature
from scipy.ndimage import distance_transform_edt, label, binary_dilation
from sklearn.datasets import make_blobs
import torch


/home/jgustafs/miniconda3/envs/adipo_env/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/jgustafs/miniconda3/envs/adipo_env/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


In [2]:
adata = sc.read('../../../data/spe_final.h5ad')

In [3]:
all_ids=adata.obs['library_id'].unique()

In [4]:
#get ground truth ids
import os
import glob
base_path = "../../../data/Manual_adipocyte_segmentation/"

# find all files ending with " step 1.tif"
files = glob.glob(os.path.join(base_path, "* step 3.tif"))

# extract IDs (everything before " step 1.tif")
gt_ids = [os.path.basename(f).replace(" step 3.tif", "") for f in files]

print(gt_ids)

['ROI001_02454_ROI_1', 'ROI001_026859_ROI_1', 'ROI001_08147_ROI_1', 'ROI001_31479_ROI_1', 'ROI001_40423_ROI_2', 'ROI001_44243_ROI_1', 'ROI001_50286_ROI_1', 'ROI001_62626_ROI_1', 'ROI001_70736_ROI_1', 'ROI001_J33533_ROI_1', 'ROI002_40900_ROI_1', 'ROI002_45504_ROI_1', 'ROI002_74014_ROI_1', 'ROI003_37912_ROI_1', 'ROI004_55218_ROI_1']


In [5]:
print(f"tot samples: {len(all_ids)}, ground truth samples: {len(gt_ids)}")

tot samples: 60, ground truth samples: 15


In [6]:
from PIL import Image
input_images = []
ground_truth_images = []
for id in all_ids:
    input_images.append(seg_utils.Preprocessing.extract_segmentation_image(adata, id))
for id in gt_ids:
    ground_truth_images.append(np.array(Image.open(base_path + id + " step 4.png")))

In [7]:
#this takes a while to run
full_df, all_bin_mask, all_distance, all_segmented, all_cleaned_mask = nac.prepare_datasets(all_ids, gt_ids, input_images, ground_truth_images, min_distance_seg_init = 20, min_island_area = 50)

Processing ROI001_50286_ROI_1
Processing ROI002_59748_ROI_1
Processing ROI001_60004_ROI_1
Processing ROI001_08147_ROI_1
Processing ROI002_11952_ROI_1
Processing ROI002_63929_ROI_1
Processing ROI001_49270_ROI_1
Processing ROI002_34234_ROI_2
Processing ROI002_08547_ROI_001
Processing ROI003_34416_ROI_3
Processing ROI001_40423_ROI_2
Processing ROI001_J33533_ROI_1
Processing ROI002_45504_ROI_1
Processing ROI001_58447_ROI_1
Processing ROI001_29469_ROI_1
Processing ROI001_47690_ROI_1
Processing ROI001_57524_ROI_1
Processing ROI003_37912_ROI_1
Processing ROI001_79790_ROI_1
Processing ROI001_76310_ROI_001
Processing ROI002_62914_ROI_1
Processing ROI001_44337_ROI_1
Processing ROI001_04011_ROI_1
Processing ROI002_44149_ROI_1
Processing ROI001_26861_ROI_1
Processing ROI002_74006_ROI_1
Processing ROI001_29095_ROI_1
Processing ROI001_55153_ROI_1
Processing ROI006_44029_ROI_1
Processing ROI001_62626_ROI_1
Processing ROI002_20945_ROI_1
Processing ROI001_65731_ROI_1
Processing ROI002_40900_ROI_1
Proce

In [8]:
full_df

,segment_id,area,eccentricity,compactness,max_dist,distance_from_shore,frac_ring_other_objects,frac_ring_tissue,ground_truth,image_id
0,1,164.420193,0.897654,0.453155,150.840976,0.0,0.690724,0.284293,0.0,ROI001_50286_ROI_1
1,2,13.638182,0.979721,0.425199,21.260292,31.0,0.785646,0.204785,0.0,ROI001_50286_ROI_1
2,3,4.242641,0.968665,0.926930,5.000000,50.0,0.700611,0.270876,0.0,ROI001_50286_ROI_1
3,4,15.779734,0.772128,0.684765,6.082763,146.0,0.005325,0.872204,0.0,ROI001_50286_ROI_1
4,5,35.846897,0.383487,0.646984,17.262677,68.0,0.519978,0.426193,0.0,ROI001_50286_ROI_1
...,...,...,...,...,...,...,...,...,...,...
77483,713,7.874008,0.581515,0.983756,5.000000,78.0,0.316151,0.663230,NaN,ROI001_73506_ROI_1
77484,714,46.346521,0.951864,0.475462,50.358713,30.0,0.760181,0.231146,NaN,ROI001_73506_ROI_1
77485,715,13.527749,0.981027,0.431882,29.120440,9.0,0.990347,0.009653,NaN,ROI001_73506_ROI_1
77486,716,18.330303,0.907965,0.655071,10.630146,85.0,0.482375,0.480519,NaN,ROI001_73506_ROI_1


In [9]:
model, train_ids, val_ids, test_ids = nac.load_model("../data/trained_model_main.pth")

In [10]:
nac.predict_and_clean_images(model, full_df, all_ids, all_segmented, 
                             gt_ids, ground_truth_images, all_bin_mask, 
                             "../../../new_predictions/", "../../../new_prediction_plots/")


Processing 1 of 60
Processing 2 of 60
Processing 3 of 60
Processing 4 of 60
Processing 5 of 60
Processing 6 of 60
Processing 7 of 60
Processing 8 of 60
Processing 9 of 60
Processing 10 of 60
Processing 11 of 60
Processing 12 of 60
Processing 13 of 60
Processing 14 of 60
Processing 15 of 60
Processing 16 of 60
Processing 17 of 60
Processing 18 of 60
Processing 19 of 60
Processing 20 of 60
Processing 21 of 60
Processing 22 of 60
Processing 23 of 60
Processing 24 of 60
Processing 25 of 60
Processing 26 of 60
Processing 27 of 60
Processing 28 of 60
Processing 29 of 60
Processing 30 of 60
Processing 31 of 60
Processing 32 of 60
Processing 33 of 60
Processing 34 of 60
Processing 35 of 60
Processing 36 of 60
Processing 37 of 60
Processing 38 of 60
Processing 39 of 60
Processing 40 of 60
Processing 41 of 60
Processing 42 of 60
Processing 43 of 60
Processing 44 of 60
Processing 45 of 60
Processing 46 of 60
Processing 47 of 60
Processing 48 of 60
Processing 49 of 60
Processing 50 of 60
Processin

In [11]:
#this is just for reloading the new_adipofinder_code in case it has been modified,
#to aviod having to restart the notebook
import importlib
import adipo_finder.new_adipofinder_code as nac

importlib.reload(nac)

ModuleNotFoundError: No module named 'new_adipofinder_code'